In [1]:
# Loading libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn
import warnings
from sklearn.neighbors import KNeighborsRegressor
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import PowerTransformer
from sklearn.metrics import f1_score
from sklearn.model_selection import cross_val_score
warnings.filterwarnings('ignore')

C:\Users\olabi\AppData\Local\Temp\ipykernel_14828\1370962977.py:3: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
data =pd.read_csv('maintenance_servicehours.csv')
display(data.head())

,BoostPressure,EngineFuelRate,EngineLoad,EngineOilPressure,EngineRpm,GroundSpeed,HaulDistance,Payload,TankFuelLevel,GearSelect,ServiceHours
0,3.12,88.19,31.7,587.67,1360.65,0.0,0.0,0.0,33.24,0.0,541.002751
1,8.23,109.86,34.6,692.82,1731.02,0.0,0.0,0.0,33.24,0.0,665.708336
2,3.05,64.07,22.3,572.30,1265.98,0.0,0.0,0.0,33.24,0.0,614.675853
3,12.03,104.47,33.5,685.22,1862.32,0.0,0.0,0.0,33.24,0.0,398.043887
4,10.08,71.29,21.4,612.62,1482.57,0.0,0.0,0.0,33.24,0.0,548.413577


In [3]:
data.shape

(11500, 11)

In [4]:
# X/y split
y = data['ServiceHours']
X = data.drop(['ServiceHours'], axis=1)

# Train/Test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state = 1)

X_train_df = pd.DataFrame(X_train, columns=X.columns)

X_test_df = pd.DataFrame(X_test, columns=X.columns)

In [5]:
# data needs to be transformed
transformer = PowerTransformer()
transformer.fit(X_train)

X_train_transformed_np = transformer.transform(X_train)
X_test_transformed_np  = transformer.transform(X_test)

X_train_transformed_df = pd.DataFrame(X_train_transformed_np)
X_test_transformed_df  = pd.DataFrame(X_test_transformed_np)

In [8]:
# Apply KNN regression: GridsearchCV: 

from sklearn.model_selection import GridSearchCV

grid = {
        'n_neighbours':[ 2, 3,4, 5 ],
         'weights':["uniform", "distance"],
         'p': [1, 2]
}
model = KNeighborsRegressor()

grid_search = GridSearchCV(estimator=model, param_grid=grid, cv=5)

In [18]:
# Define the grid of hyperparameters

grid = {
    'n_neighbors': [2, 3, 4, 5],
    'weights': ['uniform', 'distance'],
    'p': [1, 2]
}

# Create the KNeighborsRegressor model
model = KNeighborsRegressor()

# Setup GridSearchCV
grid_search = GridSearchCV(estimator=model, param_grid=grid, cv=5)

# Fit the model to the data
grid_search.fit(X_train_transformed_df, y_train)  

# Print the best hyperparameters
print("Best Hyperparameters:", grid_search.best_params_)
print("Best Estimators:", grid_search.best_estimator_)

Best Hyperparameters: {'n_neighbors': 5, 'p': 1, 'weights': 'uniform'}
Best Estimators: KNeighborsRegressor(p=1)


In [19]:
from sklearn.model_selection import cross_validate

# applying CV on the TRAIN set.
results = cross_validate(grid_search, X_train_transformed_df, y_train, cv = 5)

In [20]:
# Displaying the results
results

{'fit_time': array([2.74102521, 2.71447992, 2.72381878, 2.72672319, 2.7064817 ]),
 'score_time': array([0.04544711, 0.03318286, 0.04627156, 0.03540635, 0.0351491 ]),
 'test_score': array([-0.18555048, -0.21449902, -0.23175233, -0.18144824, -0.21440595])}

In [21]:
# result summary

print(results['test_score'])
print("The average R2 over the folds is: {:.2f}".format(results['test_score'].mean()))
print("The standard deviation of R2 over the folds is: {:.2f}".format(results['test_score'].std()))

[-0.18555048 -0.21449902 -0.23175233 -0.18144824 -0.21440595]
The average R2 over the folds is: -0.21
The standard deviation of R2 over the folds is: 0.02


# Prediction with Random forest

In [12]:
# Fitting Random Forest Regression to the train dataset GRidserachCV n_stimators=[50,100, 150], max_depth=[5, 7 ,11]; min_samples_split=[5, 10,15], cv=5
regressor = RandomForestRegressor(n_estimators=50, random_state=0, oob_score=True) 

# Fit the regressor with X_train and y_train
regressor.fit(X_train_df, y_train)

RandomForestRegressor(n_estimators=10, oob_score=True, random_state=0)

In [18]:
# Evaluating the model
from sklearn.metrics import mean_squared_error, r2_score

# Access the OOB Score
oob_score = regressor.oob_score_
print(f'Out-of-Bag Score: {oob_score}')

# Making predictions on the same data or new data
predictions = regressor.predict(X_train_df)

# Evaluating the model
mse = mean_squared_error(y_train, predictions)
print(f'Mean Squared Error: {mse: .2f}')

r2 = r2_score(y_train, predictions)
print(f'R-squared: {r2: .2f}')


Out-of-Bag Score: -0.405976476374486
Mean Squared Error:  10256.93
R-squared:  0.80


In [19]:
# Evaluating the model for the test_df
from sklearn.metrics import mean_squared_error, r2_score

# Access the OOB Score
oob_score = regressor.oob_score_
print(f'Out-of-Bag Score: {oob_score}')

# Making predictions on the same data or new data
predictions = regressor.predict(X_test_df)

# Evaluating the model
mse = mean_squared_error(y_test, predictions)
print(f'Mean Squared Error: {mse: .2f}')

r2 = r2_score(y_test, predictions)
print(f'R-squared: {r2: .2f}')


Out-of-Bag Score: -0.405976476374486
Mean Squared Error:  57672.03
R-squared: -0.14


# Predictions using KNN-Regressor

In [ ]:
grid_search = GridSearchCV(estimator=model, param_grid=grid, cv=5)

In [14]:
# Apply KNN regression: GridsearchCV: 
grid = {
        'n_neighbours':[ 2, 3,´4, 5 ],
         'weights':["uniform", "distance"], 
         'cv':5
}
model = KNeighborsRegressor()

grid_search = GridSearchCV(estimator=model, param_grid=grid, cv=5)

knn_regressor.fit(X_train_df, y_train)
predictions = knn_regressor.predict(X_test)
 
# Evaluate the model
print('R2 Score:', knn_regressor.score(X_test, y_test))

Score: -0.9643423758821434


In [15]:
#Measuring Goodness of fit in Training data
from sklearn import metrics
print('R2 Value:',metrics.r2_score(y_train, knn_regressor.predict(X_train_df)))
 
#Measuring accuracy on Testing Data
print('Accuracy',100- (np.mean(np.abs((y_test - predictions) / y_test)) * 100))
 


R2 Value: 1.0
Accuracy -37.720771160710655


In [ ]:
predictions = knn_regressor.predict(X_test)
 
# Evaluate the model
print('R2 Score:', knn_regressor.score(X_test, y_test))